In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import trimesh
import osmium
import re
import warnings
import mapbox_earcut as earcut
import matplotlib.pyplot as plt

from tqdm.notebook import trange, tqdm
from shapely.ops import split, snap, unary_union, polygonize, linemerge
from shapely.geometry import MultiPolygon, Polygon, MultiLineString, LineString, Point, MultiPoint, box, polygon
from pyproj import Transformer

warnings.simplefilter(action='ignore', category=FutureWarning)

In [22]:
class RelationHandler(osmium.SimpleHandler):

    def __init__(self):
        osmium.SimpleHandler.__init__(self)
        self.count_bid = 0
        self.area_to_bid = {}
        self.relation_to_bid = {}

    def get_area_to_bid(self):
        return self.area_to_bid
    
    def get_relation_to_bid(self):
        return self.relation_to_bid
        
    def relation(self, r):
        tags = dict(r.tags)
        
        # Qualifiers
        if not ('building' in tags or 'building:part' in tags or tags.get('type') == 'building'):
            return
        # Disqualifiers
        if (tags.get('location') == 'underground' or 'bridge' in tags):
            return
        
        if r.id not in self.relation_to_bid:
            self.relation_to_bid[r.id] = self.count_bid
            self.count_bid +=1
        
        for member in r.members:
            if member.ref not in self.area_to_bid:
                self.area_to_bid[member.ref] = self.relation_to_bid[r.id]

class AreaHandler(osmium.SimpleHandler):

    def __init__(self, area_to_bid, relation_to_bid):
        osmium.SimpleHandler.__init__(self)
        self.id = []
        self.orig_id = []
        self.building_id = []
        self.tag = []
        self.geometry = []
        self.height = []
        self.min_height = []
        self.wkbfab = osmium.geom.WKBFactory()
        
        self.area_to_bid = area_to_bid
        self.relation_to_bid = relation_to_bid
        self.max_bid = max(self.area_to_bid, key=area_to_bid.get)
        self.max_bid = max(self.max_bid,max(self.relation_to_bid, key=relation_to_bid.get))

        self.LEVEL_HEIGHT = 3.4

    # https://wiki.openstreetmap.org/wiki/Simple_3D_buildings#Other_roof_tags
    def _feet_to_meters(self, s):
        r = re.compile("([0-9]*\.?[0-9]+)'([0-9]*\.?[0-9]+)?\"?")
        m = r.findall(s)[0]
        if len(m[0]) > 0 and len(m[1]) > 0:
            m = float(m[0]) + float(m[1]) / 12.0
        elif len(m[0]) > 0:
            m = float(m[0])
        return m * 0.3048

    def _get_height(self, tags):
        if 'height' in tags:
            # already accounts for roof
            if '\'' in tags['height'] or '\"' in tags['height']:
                return self._feet_to_meters(tags['height'])
            r = re.compile(r"[-+]?\d*\.\d+|\d+")
            return float(r.findall(tags['height'])[0])
        if 'levels' in tags:
            roof_height = 0
            if 'roof_height' in tags:
                if '\'' in tags['roof_height'] or '\"' in tags['roof_height']:
                    roof_height = self._feet_to_meters(tags['roof_height'])
                else:
                    r = re.compile(r"[-+]?\d*\.\d+|\d+")
                    roof_height = float(r.findall(tags['roof_height'])[0])

            # does not account for roof height
            height = float(tags['levels']) * self.LEVEL_HEIGHT
            if 'roof_levels' in tags and roof_height == 0:
                height += float(tags['roof_levels']) * self.LEVEL_HEIGHT
            return height
        return 7.0

    def _get_min_height(self, tags):
        if 'min_height' in tags:
            # already accounts for roof
            if '\'' in tags['min_height'] or '\"' in tags['min_height']:
                return self._feet_to_meters(tags['min_height'])
            r = re.compile(r"[-+]?\d*\.\d+|\d+")
            return float(r.findall(tags['min_height'])[0])
        if 'min_level' in tags:
            height = float(tags['min_level']) * self.LEVEL_HEIGHT
            return height
        return 0.0
        
    def get_gdf(self):
        geometry = gpd.GeoSeries.from_wkb(self.geometry, crs='epsg:4326')
        height = pd.Series(self.height, dtype='float')
        min_height = pd.Series(self.min_height, dtype='float')
        tag = pd.Series(self.tag)
        iid = pd.Series(self.id, dtype='UInt64')
        orig_id = pd.Series(self.orig_id, dtype='UInt64')
        building_id = pd.Series(self.building_id, dtype='UInt64')
        
        return gpd.GeoDataFrame({
            'id': iid,
            'orig_id': orig_id,
            'building_id': building_id,
            'geometry': geometry,
            'min_height': min_height,
            'height': height,
            'tags': tag
        }, index=geometry.index)
    
    def area(self, a):
        tags = dict(a.tags)
        iid = int(a.id)
        orig_id = int(a.orig_id())
        
        # Qualifiers
        if not ('building' in tags or 'building:part' in tags or tags.get('type') == 'building'):
            return
        # Disqualifiers
        if (tags.get('location') == 'underground' or 'bridge' in tags):
            return
        
        if orig_id in self.area_to_bid:
            building_id = self.area_to_bid[orig_id]
        elif orig_id in self.relation_to_bid:
            building_id = self.relation_to_bid[orig_id]
        else:
            building_id = self.max_bid
            self.max_bid+=1
        
        try:
            poly = self.wkbfab.create_multipolygon(a)
            height = self._get_height(tags)
            min_height = self._get_min_height(tags)
            
            self.geometry.append(poly)
            self.height.append(height)
            self.min_height.append(min_height)
            self.tag.append(tags)
            self.id.append(iid)
            self.orig_id.append(orig_id)
            self.building_id.append(building_id)
            
        except Exception as e:
            print(e)
            print(a)

In [3]:
def split_poly(poly, size):
    merged_segments = []
    
    boundaries = list(poly)
    for poly in boundaries:
        lines = poly.exterior
        length = lines.length
        size = min(size, length)
        num_cells = ((length - length%size)/size) # equal sized cells
        size = length / num_cells
        distances = np.arange(0, length, size)

        interpolated_points = list(MultiPoint([lines.interpolate(distance) for distance in distances]))
        points = list(MultiPoint(lines.coords))

        joined_points = []

        for j in range(0, len(points)-1):
            segment = LineString([points[j], points[j+1]])
            joined_points.append(points[j])

            aux = []
            for k in range(0, len(interpolated_points)):
                pt = interpolated_points[k]
                if pt.buffer(1e-8).intersects(segment):
                    joined_points.append(pt)
                else:
                    aux.append(pt)
            interpolated_points = aux 

        joined_points.append(points[-1])

        cur_length = 0
        segments = []
        cur_multiline = []
        for j in range(0, len(joined_points)-1):
            segment = LineString([joined_points[j], joined_points[j+1]])
            cur_multiline.append(segment)
            cur_length += segment.length

            if (cur_length >= size-1e-5):
                cur_length = 0
                cur_pt = joined_points[j]
                segments.append(cur_multiline)
                cur_multiline = []

#             if (cur_length < DISTANCE_DELTA-1e-5) and (j == len(joined_points)-2):
#                 segments[-1].append(segment)


        for j in range(0,len(segments)):
            seg = linemerge(MultiLineString(segments[j]))
            merged_segments.append(seg)
        
    return merged_segments

def extrude(segments, min_height, height, size):
    
    length = height-min_height
    size = min(size, length)
    num_cells = ((length - length%size)/size) # equal sized cells

    size = length/num_cells
    distances = np.arange(0, length, size)
    
    coordinates = []
    indices = []
    ids = []
    colors = []
    count = 0
    for seg in segments:
        count+=1
        for distance in distances:
            points = list(zip(*seg.coords.xy))
            color = np.random.rand(3,)
            for i in range(0, len(points)-1):
                
                p0 = points[i]
                p1 = points[i+1]

                v0 = [p0[0],p0[1],distance+min_height]
                v1 = [p1[0],p1[1],distance+min_height]
                v2 = [p0[0],p0[1],distance+min_height+size]
                v3 = [p1[0],p1[1],distance+min_height+size]

                i0 = int(len(coordinates)/3)
                coordinates.extend(v0)
                coordinates.extend(v1)
                coordinates.extend(v2)
                coordinates.extend(v3)
                
                colors.append(color)
                colors.append(color)
                colors.append(color)
                colors.append(color)

                # t0
                indices.append(i0)
                indices.append(i0+1)
                indices.append(i0+2)

                # t1
                indices.append(i0+1)
                indices.append(i0+3)
                indices.append(i0+2)
    
    coordinates = np.array(coordinates).reshape(-1, 3)
    indices = np.array(indices).reshape(-1, 3)
    colors = np.array(colors).reshape(-1, 3)
    return coordinates, indices, ids, colors

def get_roof(poly, height, size):
    gdf_poly = gpd.GeoDataFrame(geometry=[poly])
    
    coordinates = []
    indices = []
    ids = []
    colors = []
    
    for poly in list(gdf_poly.boundary[0].geoms):
    
        line = LineString(poly.coords)
        gdf_line = gpd.GeoDataFrame(geometry=[line])
    
        xmin, ymin, xmax, ymax= gdf_poly.total_bounds

        cell_width = cell_height = size
        cells = []
        for x0 in np.arange(xmin, xmax+cell_width, cell_width):
            for y0 in np.arange(ymin, ymax+cell_height, cell_height):
                x1 = x0-cell_width
                y1 = y0+cell_height
                new_cell = box(x0, y0, x1, y1)
                if gdf_poly.intersects(new_cell).all():
                    intersect = gdf_poly.intersection(new_cell)[0]
                    if intersect.geom_type == 'MultiPolygon':
                        intersect = list(intersect)
                        cells.extend(intersect)
                    elif intersect.geom_type == 'Polygon':
                        cells.append(intersect)

        count = 0
        for cell in cells:
            points = np.array(cell.exterior.coords)
            rings = np.array([len(points)])
            ind = (earcut.triangulate_float64(points, rings)+count-1).tolist()
            indices = indices + ind

            color = np.random.rand(3,)
            points = points.flatten().tolist()
            for i in range(0, len(points), 2):
                coordinates.append(points[i])
                coordinates.append(points[i+1])
                coordinates.append(height)
                colors.append(color)
    #         break
            count = int(len(coordinates)/3)
        
    
    coordinates = np.array(coordinates).reshape(-1, 3)
    indices = np.array(indices).reshape(-1, 3)
    colors = np.array(colors).reshape(-1, 3)
        
    return coordinates, indices, ids, colors

def merge_building(building):
    min_heights = building.min_height.values
    heights = building.height.values
    all_heights = np.concatenate((min_heights, heights))
    all_heights = np.unique(all_heights)
    all_heights = np.sort(all_heights)
    
    merged_geometry = []
    merged_height = []
    merged_min_height = []
    for i in range(0, len(all_heights)-1):
        min_height = all_heights[i]
        height = all_heights[i+1]
        bbl = building[(building['min_height'] <= all_heights[i]) & (building['height'] > all_heights[i])]
        merged_geom = bbl.unary_union
        if merged_geom.type == 'Polygon':
            merged_geom = MultiPolygon([merged_geom])

        oriented_geom = []
        for geom in merged_geom:
            oriented_geom.append(polygon.orient(geom, 1)) # counter-clockwise
        oriented_geom = MultiPolygon(oriented_geom)

        merged_geometry.append(oriented_geom)
        merged_min_height.append(min_height)
        merged_height.append(height)
        
    d = {'min_height': merged_min_height, 'height': merged_height, 'geometry': merged_geometry}
    merged_building = gpd.GeoDataFrame(d, crs='epsg:3395')
    merged_building['building_id'] = building.iloc[0]['building_id']
    
    return merged_building

def get_mesh(building, size):
    merged_building = merge_building(building)
    boundaries = list(merged_building.geometry)

    coords = np.empty((0,3))
    indices = np.empty((0,3))
    ids = np.empty((0,1))
    colors = np.empty((0,3))

    for i in range(0, len(boundaries)):
        geom = boundaries[i].geoms
        height = merged_building.iloc[i].height
        min_height = merged_building.iloc[i].min_height

        # roof
        coordinates_roof, indices_roof, ids_roof, colors_roof = get_roof(boundaries[i], height, size)
        coordinates_roof = coordinates_roof.reshape(-1, 3)
        indices_roof = indices_roof.reshape(-1, 3)
        indices_roof = indices_roof + coords.shape[0]

        # walls
        segments = split_poly(geom, size)
        coordinates_walls, indices_walls, ids_walls, colors_walls = extrude(segments, min_height, height, size)
        indices_walls = indices_walls + coordinates_roof.shape[0] + coords.shape[0]

        coordinates_walls = coordinates_walls.reshape(-1, 3)
        indices_walls = indices_walls.reshape(-1, 3)

        coords = np.concatenate((coords, coordinates_roof, coordinates_walls))
        indices = np.concatenate((indices, indices_roof, indices_walls))
    #     ids = np.concatenate((ids, ids_roof, ids_walls))
        colors = np.concatenate((colors, colors_roof, colors_walls))

    coords = coords.reshape(-1, 3)
    indices = indices.reshape(-1, 3)
    
    return coords, indices, ids, colors

In [23]:
h = RelationHandler()
h.apply_file('data/nyc.osm.pbf', locations=True)
area_to_bid = h.get_area_to_bid()
relation_to_bid = h.get_relation_to_bid()

In [24]:
h = AreaHandler(area_to_bid, relation_to_bid)
h.apply_file('data/nyc.osm.pbf', locations=True)

invalid area (area_id=533866792)
osmium.osm.Area(id=533866792, deleted=False, visible=True, version=2, changeset=0, uid=0, timestamp=datetime.datetime(2022, 3, 14, 23, 26, 8, tzinfo=datetime.timezone.utc), user='', tags=osmium.osm.TagList({'addr:housenumber': '178', 'addr:postcode': '10128', 'addr:street': 'East 88th Street', 'building': 'yes', 'height': '15.4', 'nycdoitt:bin': '1048058'}))


In [25]:
gdf = h.get_gdf()
gdf = gdf.to_crs('epsg:3395')

In [36]:
# for i in trange(len(gdf)):
    
#     break

In [33]:
building

,id,orig_id,building_id,geometry,min_height,height,tags
47,69267708,34633854,7,"MULTIPOLYGON (((-8236137.891 4947416.484, -823...",0.0,443.2,"{'addr:city': 'New York', 'addr:housenumber': ..."
714,274850250,137425125,7,"MULTIPOLYGON (((-8236082.632 4947405.360, -823...",290.0,310.0,"{'building:colour': 'beige', 'building:part': ..."
715,274850254,137425127,7,"MULTIPOLYGON (((-8236093.019 4947401.832, -823...",115.0,255.0,"{'building:colour': 'beige', 'building:levels'..."
716,274850256,137425128,7,"MULTIPOLYGON (((-8236095.679 4947413.337, -823...",90.0,115.0,"{'building:colour': 'beige', 'building:levels'..."
717,274850258,137425129,7,"MULTIPOLYGON (((-8236089.412 4947402.828, -823...",255.0,290.0,"{'building:colour': 'beige', 'building:levels'..."
718,21744109,10872054,7,"MULTIPOLYGON (((-8236137.891 4947416.484, -823...",0.0,17.0,"{'building:colour': 'beige', 'building:levels'..."
719,274850262,137425131,7,"MULTIPOLYGON (((-8236114.704 4947413.938, -823...",20.0,75.0,"{'building:colour': 'beige', 'building:levels'..."
720,274850266,137425133,7,"MULTIPOLYGON (((-8236104.830 4947401.159, -823...",75.0,90.0,"{'building:colour': 'beige', 'building:levels'..."
721,274850284,137425142,7,"MULTIPOLYGON (((-8236075.441 4947407.365, -823...",0.0,330.0,"{'building:colour': 'beige', 'building:part': ..."
722,274850288,137425144,7,"MULTIPOLYGON (((-8236080.829 4947405.858, -823...",0.0,320.0,"{'building:colour': 'beige', 'building:part': ..."


In [35]:
# Test one building
building = gdf[gdf['building_id']==40]
coordinates, indices, ids, colors = get_mesh(building, 5)

mesh = trimesh.Trimesh(vertices = coordinates, faces = indices, vertex_colors = colors, process= False, fix_normals=True)
translation = trimesh.transformations.translation_matrix(-mesh.centroid)
mesh = mesh.apply_transform(translation)
mesh.show(smooth=False)